In [34]:
! pip install mlxtend

In [110]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import MinMaxScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from mlxtend.feature_selection import SequentialFeatureSelector as SFS

In [111]:
df_train=pd.read_csv("house_price_train.csv")
df_train.drop("Id",axis=1,inplace=True) ## reduntatnt
output_col=df_train['SalePrice']
df_train=df_train.drop('SalePrice',axis=1)

In [112]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [113]:
df_train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
Length: 79, dtype: object

Checking Negative values.

In [114]:
storing_count={}
for col in df_train.columns:
    if df_train[col].dtypes=='int' or df_train[col].dtypes=='float':
        c=0
        for val in df_train[col]:
            if val<0:
                c+=1
        storing_count[col]=c
storing_count

{'MSSubClass': 0,
 'LotFrontage': 0,
 'LotArea': 0,
 'OverallQual': 0,
 'OverallCond': 0,
 'YearBuilt': 0,
 'YearRemodAdd': 0,
 'MasVnrArea': 0,
 'BsmtFinSF1': 0,
 'BsmtFinSF2': 0,
 'BsmtUnfSF': 0,
 'TotalBsmtSF': 0,
 '1stFlrSF': 0,
 '2ndFlrSF': 0,
 'LowQualFinSF': 0,
 'GrLivArea': 0,
 'BsmtFullBath': 0,
 'BsmtHalfBath': 0,
 'FullBath': 0,
 'HalfBath': 0,
 'BedroomAbvGr': 0,
 'KitchenAbvGr': 0,
 'TotRmsAbvGrd': 0,
 'Fireplaces': 0,
 'GarageYrBlt': 0,
 'GarageCars': 0,
 'GarageArea': 0,
 'WoodDeckSF': 0,
 'OpenPorchSF': 0,
 'EnclosedPorch': 0,
 '3SsnPorch': 0,
 'ScreenPorch': 0,
 'PoolArea': 0,
 'MiscVal': 0,
 'MoSold': 0,
 'YrSold': 0}

We got 0 negative values in all the numerical columns. For distribution we will use describe method .

In [115]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1379.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,443.639726,567.240411,1057.429452,...,2.866438,1.046575,6.517808,0.613014,1978.506164,1.767123,472.980137,46.660274,6.321918,2007.815753
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,456.098091,441.866955,438.705324,...,0.815778,0.220338,1.625393,0.644666,24.689725,0.747315,213.804841,66.256028,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,223.000000,795.750000,...,2.000000,1.000000,5.000000,0.000000,1961.000000,1.000000,334.500000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,383.500000,477.500000,991.500000,...,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,25.000000,6.000000,2008.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,712.250000,808.000000,1298.250000,...,3.000000,1.000000,7.000000,1.000000,2002.000000,2.000000,576.000000,68.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,5644.000000,2336.000000,6110.000000,...,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,547.000000,12.000000,2010.000000


We can infer from above tables that there are columns such as EnclosedPorch which have 75% of their values smaller than 0 but there were no negative values as we already have seen. So all together we can say that there are 75% values which are 0. So this is leading to unbalanced dataset thats why we need to delete these columns

In [116]:
# Columns which have zero values in more than 50% rows are removed
def column_with_most_reduntant_values(reduntant_value,df_train:pd.DataFrame,threshold:float):
    reduntant_counts={}
    columns_with_most_reduntant_counts=[]
    for col in df_train.columns:
        if df_train[col].dtypes=='int' or df_train[col].dtypes=='float':
            for values in df_train[col]:
                if values==reduntant_value:
                    reduntant_counts[col]=1+reduntant_counts.get(col,0)
    for col in reduntant_counts:
        if reduntant_counts[col]>=(threshold*len(df_train)):
            columns_with_most_reduntant_counts.append(col)
    df_train.drop(columns_with_most_reduntant_counts,axis=1,inplace=True)

In [117]:
column_with_most_reduntant_values(reduntant_value=0,df_train=df_train,threshold=0.5)

In [118]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,GarageCond,PavedDrive,OpenPorchSF,PoolQC,Fence,MiscFeature,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,61,NaN,NaN,NaN,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,TA,Y,0,NaN,NaN,NaN,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,TA,Y,42,NaN,NaN,NaN,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,TA,Y,35,NaN,NaN,NaN,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,TA,Y,84,NaN,NaN,NaN,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,40,NaN,NaN,NaN,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,MnPrv,NaN,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,60,NaN,GdPrv,Shed,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,TA,Y,0,NaN,NaN,NaN,4,2010,WD,Normal


In [119]:

def column_with_most_reduntant_null_values(threshold,df_train:pd.DataFrame):
    null_cols=[]
    for col in df_train.columns:
        if df_train[col].isnull().sum()>=(threshold*len(df_train)):
            null_cols.append(col)
    df_train.drop(null_cols,axis=1,inplace=True)

In [120]:
column_with_most_reduntant_null_values(threshold=0.5,df_train=df_train)

In [121]:
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,OpenPorchSF,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,548,TA,TA,Y,61,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,2,460,TA,TA,Y,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,2,608,TA,TA,Y,42,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,3,642,TA,TA,Y,35,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,3,836,TA,TA,Y,84,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,460,TA,TA,Y,40,8,2007,WD,Normal
1456,20,RL,85.0,13175,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,2,500,TA,TA,Y,0,2,2010,WD,Normal
1457,70,RL,66.0,9042,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,252,TA,TA,Y,60,5,2010,WD,Normal
1458,20,RL,68.0,9717,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,1,240,TA,TA,Y,0,4,2010,WD,Normal


In [122]:
df_train.columns

Index(['MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope',
       'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtUnfSF',
       'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical',
       '1stFlrSF', 'GrLivArea', 'FullBath', 'BedroomAbvGr', 'KitchenAbvGr',
       'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces',
       'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish',
       'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive',
       'OpenPorchSF', 'MoSold', 'YrSold', 'SaleType', 'SaleCondition'],
      dtype='object')

In [123]:
i=0
for cols in df_train.columns:
    if df_train[cols].dtype=="int64":
        print(cols,i)
#         i+=1
    elif df_train[cols].dtype=="object":
#         print(cols,i)
        i+=1
    elif df_train[cols].dtype=="float64":
        print(cols,i)
#         i+=1
print(i)

MSSubClass 0
LotFrontage 1
LotArea 1
OverallQual 12
OverallCond 12
YearBuilt 12
YearRemodAdd 12
BsmtFinSF1 24
BsmtUnfSF 25
TotalBsmtSF 25
1stFlrSF 29
GrLivArea 29
FullBath 29
BedroomAbvGr 29
KitchenAbvGr 29
TotRmsAbvGrd 30
Fireplaces 31
GarageYrBlt 33
GarageCars 34
GarageArea 34
OpenPorchSF 37
MoSold 37
YrSold 37
39


In [124]:
def fill_null_values(for_numerical_cols:str,for_categorical_cols:str,df_train):
    for cols in df_train.columns:
        if df_train[cols].dtype=="int64" or df_train[cols].dtype=="float64":
            if for_numerical_cols=="median":
                median_=df_train[cols].median()
                df_train[cols].fillna(median_,inplace=True)
        elif df_train[cols].dtype=="object":
            if for_categorical_cols=="most_frequent":
                d=list(df_train[cols].value_counts().index) # most frequent
                df_train[cols].fillna(d[0],inplace=True)

In [125]:
fill_null_values(for_numerical_cols="median",for_categorical_cols="most_frequent",df_train=df_train)

In [126]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>(0):

        print(col,df[col].isnull().sum(),df_train[col].dtypes)


In [127]:
categorical_cols=list(df_train.select_dtypes(exclude= np.number).columns)

In [128]:
continuous_cols=list(df_train.select_dtypes(include= np.number).columns)

In [129]:
continuous_cols

['MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'BsmtFinSF1',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'FullBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'OpenPorchSF',
 'MoSold',
 'YrSold']

In [130]:
categorical_cols

['MSZoning',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'SaleType',
 'SaleCondition']

In [131]:
# fig, axes = plt.subplots(nrows = 12, ncols = 3)    # axes is 2d array (3x3)
# axes = axes.flatten()         # Convert axes to 1d array of length 9
# fig.set_size_inches(20, 20)

# for ax, col in zip(axes, continuous_cols):
#     print()
#     sns.boxplot(df_train[col], ax = ax)
#     ax.set_title(col)

In [132]:
# plt.figure(figsize=(20,15))
# sns.heatmap(df_train.loc[:,continuous_cols].corr(method='spearman'),annot=True,cmap="RdYlGn")
# plt.show()

In [138]:
def encoding_categorical_cols(df_train:pd.DataFrame,categorical_columns:list,strategy:str):
    df_categorical=pd.DataFrame(df_train.loc[:,categorical_columns])
    if strategy=="labelencoder":
        for col in list(df_categorical.columns):
            cat=[]
            d={}

            cat.append([df_categorical[col].value_counts().index,len(df_categorical[col].value_counts().index)])

            for j in range(int(cat[0][1])):
                d[str(cat[0][0][j])]=j
            df_categorical[col]=df_categorical[col].map(d)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        
        df_train=pd.concat([df_train,df_categorical],axis=1)
        
    if strategy=="onehotencoder":
        one_hot_encoded=pd.get_dummies(df_categorical)
        # print(one_hot_encoded)
        df_train.drop(categorical_columns,axis=1,inplace=True)
        df_train=pd.concat([df_train,one_hot_encoded],axis=1)
    return df_train

In [139]:
df_train=encoding_categorical_cols(df_train=df_train,categorical_columns=categorical_cols,strategy="labelencoder")

In [140]:
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,60,65.0,8450,7,5,2003,2003,706,150,856,...,1,0,0,0,1,0,0,0,0,0
1,20,80.0,9600,6,8,1976,1976,978,284,1262,...,0,0,1,0,1,0,0,0,0,0
2,60,68.0,11250,7,5,2001,2002,486,434,920,...,1,0,1,0,1,0,0,0,0,0
3,70,60.0,9550,7,5,1915,1970,216,540,756,...,1,0,0,1,0,0,0,0,0,2
4,60,84.0,14260,8,5,2000,2000,655,490,1145,...,1,0,1,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,62.0,7917,6,5,1999,2000,0,953,953,...,0,0,1,0,1,0,0,0,0,0
1456,20,85.0,13175,6,6,1978,1988,790,589,1542,...,0,2,1,0,0,0,0,0,0,0
1457,70,66.0,9042,7,9,1941,2006,275,877,1152,...,1,0,0,0,1,0,0,0,0,0
1458,20,68.0,9717,5,6,1950,1996,49,0,1078,...,1,0,0,0,0,0,0,0,0,0


In [141]:
df_train.loc[:,continuous_cols].describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,OpenPorchSF,MoSold,YrSold
count,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,69.863699,10516.828082,6.099315,5.575342,1971.267808,1984.865753,443.639726,567.240411,1057.429452,...,2.866438,1.046575,6.517808,0.613014,1978.589041,1.767123,472.980137,46.660274,6.321918,2007.815753
std,42.300571,22.027677,9981.264932,1.382997,1.112799,30.202904,20.645407,456.098091,441.866955,438.705324,...,0.815778,0.220338,1.625393,0.644666,23.997022,0.747315,213.804841,66.256028,2.703626,1.328095
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,1900.000000,0.000000,0.000000,0.000000,1.000000,2006.000000
25%,20.000000,60.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,223.000000,795.750000,...,2.000000,1.000000,5.000000,0.000000,1962.000000,1.000000,334.500000,0.000000,5.000000,2007.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,383.500000,477.500000,991.500000,...,3.000000,1.000000,6.000000,1.000000,1980.000000,2.000000,480.000000,25.000000,6.000000,2008.000000
75%,70.000000,79.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,712.250000,808.000000,1298.250000,...,3.000000,1.000000,7.000000,1.000000,2001.000000,2.000000,576.000000,68.000000,8.000000,2009.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,5644.000000,2336.000000,6110.000000,...,8.000000,3.000000,14.000000,3.000000,2010.000000,4.000000,1418.000000,547.000000,12.000000,2010.000000


In [142]:
for col in df_train.columns:
    if df_train[col].isnull().sum()>0:
        print(col)

In [143]:
### SCALING

In [144]:
scaler=scaler=MinMaxScaler()
df_train=pd.DataFrame(scaler.fit_transform(df_train),columns=df_train.columns,)
df_train

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtUnfSF,TotalBsmtSF,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,0.235294,0.150685,0.033420,0.666667,0.500,0.949275,0.883333,0.125089,0.064212,0.140098,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.202055,0.038795,0.555556,0.875,0.753623,0.433333,0.173281,0.121575,0.206547,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
2,0.235294,0.160959,0.046507,0.666667,0.500,0.934783,0.866667,0.086109,0.185788,0.150573,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
3,0.294118,0.133562,0.038561,0.666667,0.500,0.311594,0.333333,0.038271,0.231164,0.123732,...,0.333333,0.000000,0.00,0.2,0.0,0.0,0.0,0.0,0.0,0.4
4,0.235294,0.215753,0.060576,0.777778,0.500,0.927536,0.833333,0.116052,0.209760,0.187398,...,0.333333,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,0.235294,0.140411,0.030929,0.555556,0.500,0.920290,0.833333,0.000000,0.407962,0.155974,...,0.000000,0.000000,0.25,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1456,0.000000,0.219178,0.055505,0.555556,0.625,0.768116,0.633333,0.139972,0.252140,0.252373,...,0.000000,0.333333,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,0.294118,0.154110,0.036187,0.666667,1.000,0.500000,0.933333,0.048724,0.375428,0.188543,...,0.333333,0.000000,0.00,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1458,0.000000,0.160959,0.039342,0.444444,0.625,0.565217,0.766667,0.008682,0.000000,0.176432,...,0.333333,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [145]:
from sklearn.metrics import r2_score

In [146]:
X=df_train
Y=output_col

In [147]:
accuracies={}

In [148]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=None,shuffle=True)


In [154]:
def sf_selector(algo:str,forward:bool,floating:bool,scoring:int,X_train,Y_train,X_test,Y_test,min_featues:int,cv:int,verbose:int):
    model=eval(algo)
    sfs=SFS(model, 
           k_features=(min_featues,len(X_train.columns)), 
           forward=forward, 
           floating=floating, 
           verbose=verbose,
           scoring=scoring,
           cv=cv,
           n_jobs=-1)
    sfs=sfs.fit(X_train, Y_train)
    print("Selected Features",sfs.k_feature_names_)
    model_=eval(algo)
    model_.fit(X_train.loc[:,list(sfs.k_feature_names_)],Y_train)
    score=model_.score(X_test.loc[:,list(sfs.k_feature_names_)],Y_test)
    
    print(f"Prediction using selected features using {model_} is :",score)
    accuracies[model]=score

In [150]:
sf_selector(algo="LinearRegression()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    1.2s finished

[2023-02-01 14:11:04] Features: 1/62 -- score: 0.6324391457008305[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    0.1s finished

[2023-02-01 14:11:04] Features: 2/62 -- score: 0.7291035186272596[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    0.1s finished

[2023-02-01 14:11:04] Features: 3/62 -- score: 0.7551150057927062[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  41 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  59 out of  59 | elapsed:    0.1s finished

[2023-02-01 14:11:04] Features: 4/62 -- score: 0.7723432652912546[Parallel(n_jobs=-1)]: U

Selected Features ('MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'BsmtFinSF1', 'GrLivArea', 'KitchenAbvGr', 'Fireplaces', 'GarageCars', 'MSZoning', 'LandSlope', 'Neighborhood', 'Condition1', 'RoofStyle', 'MasVnrType', 'ExterQual', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'Functional', 'SaleType')
Prediction using selected features using LinearRegression() is : 0.7740871923594853


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished

[2023-02-01 14:11:10] Features: 58/62 -- score: 0.8023099675797687[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    0.0s finished

[2023-02-01 14:11:11] Features: 59/62 -- score: 0.7984091315124944[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    0.0s finished

[2023-02-01 14:11:11] Features: 60/62 -- score: 0.7906927053963321[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    0.0s finished

[2023-02-01 14:11:11] Features: 61/62 -- score: 0.7897857101914614[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s f

In [ ]:
sf_selector(algo="RandomForestRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    7.2s finished

[2023-02-01 14:04:09] Features: 1/62 -- score: 0.6715718275356732[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    6.6s finished

[2023-02-01 14:04:16] Features: 2/62 -- score: 0.7426257932637758[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    7.8s finished

[2023-02-01 14:04:24] Features: 3/62 -- score: 0.797156733577357[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    5.8s


In [155]:
sf_selector(algo="SVR()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  62 out of  62 | elapsed:    2.7s finished

[2023-02-01 14:14:19] Features: 1/62 -- score: -0.053422661398255133[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  61 out of  61 | elapsed:    2.9s finished

[2023-02-01 14:14:22] Features: 2/62 -- score: -0.05353311011256982[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:    2.7s finished

[2023-02-01 14:14:25] Features: 3/62 -- score: -0.05362870137801061[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done  5

Selected Features None



STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

TypeError: 'NoneType' object is not iterable

In [ ]:
sf_selector(algo="KNeighborsRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="DecisionTreeRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="SGDRegressor()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="ElasticNet()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="Lasso()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [ ]:
sf_selector(algo="Ridge()",forward=True,floating=False,scoring="r2",X_train=X_train,Y_train=Y_train,X_test=X_test,Y_test=Y_test,min_featues=3,cv=5,verbose=2)

In [75]:

# sfs1 = SFS(LinearRegression(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=0,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()
# model_=LinearRegression()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["LinearRegression"]=score


sfs1 max score features are : ('MSSubClass', 'OverallQual', 'OverallCond', 'YearBuilt', 'BsmtUnfSF', 'GrLivArea', 'Fireplaces', 'GarageCars', 'MSZoning', 'LandContour', 'LandSlope', 'Neighborhood', 'Condition1', 'BldgType', 'RoofStyle', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'GarageFinish', 'PavedDrive')
sfs1 max score is : 0.7809731892848324

Prediction using selected features is : 0.8041558213003999


In [6]:

# sfs1 = SFS(RandomForestRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [7]:
# model_=RandomForestRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["RandomForestRegressor"]=score

In [8]:
# sfs1 = SFS(SVR(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [9]:
# model_=SVR()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SVR"]=score

In [10]:
# sfs1 = SFS(KNeighborsRegressor(), 
#            k_features=(3,len(X_train.columns)), 
#            forward=True, 
#            floating=False, 
#            verbose=1,
#            scoring='r2',
#            cv=5,
#            n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [11]:
# model_=KNeighborsRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["KNeighborsRegressor"]=score

In [12]:
# sfs1 = SFS(DecisionTreeRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [13]:
# model_=DecisionTreeRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["DecisionTreeRegressor"]=score

In [14]:
# sfs1 = SFS(SGDRegressor(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [15]:
# model_=SGDRegressor()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["SGDRegressor"]=score

In [16]:
# sfs1 = SFS(ElasticNet(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [17]:
# model_=ElasticNet()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["ElasticNet"]=score

In [18]:
# sfs1 = SFS(Lasso(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [19]:
# model_=Lasso()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Lasso"]=score

In [20]:
# sfs1 = SFS(Ridge(), 
#                k_features=(3,len(X_train.columns)), 
#                forward=True, 
#                floating=False, 
#                verbose=1,
#                scoring='r2',
#                cv=5,
#                n_jobs=-1)
# sfs1=sfs1.fit(X_train, Y_train)
# print("sfs1 max score features are :",sfs1.k_feature_names_)
# print("sfs1 max score is :",sfs1.k_score_)
# print()


In [21]:
# model_=Ridge()
# model_.fit(X_train.loc[:,list(sfs1.k_feature_names_)],Y_train)
# score=model_.score(X_test.loc[:,list(sfs1.k_feature_names_)],Y_test)
# print("Prediction using selected features is :",score)
# accuracies["Ridge"]=score 

In [76]:
accuracies

{'LinearRegression': 0.8041558213003999,
 'RandomForestRegressor': 0.8346437069075182,
 'SVR': 0.8306893722831081,
 'KNeighborsRegressor': 0.7653386571998051,
 'DecisionTreeRegressor': 0.7407027433860756,
 'SGDRegressor': 0.7376309839088498,
 'ElasticNet': -0.006400048042944917,
 'Lasso': -0.006400048042944917,
 'Ridge': 0.7990553883773874}